In [ ]:
import pandas as pd
import numpy as np
import synapseclient
import pylab as plt
import seaborn as sns
import scipy.stats as stats
import ipywidgets as widgets
# %matplotlib widget #breaks javascript in notebook

import create_summary

COMPLETION_COLS = ['wasCompleted', 'inSynapseParent', 'inStudyProject', 'inParquet', 'inScores']
DELTA_T_COLS = ['completion time [s]', 'export delay [s]', 'upload delay [s]', 'in Synapse delay [s]']

syn = synapseclient.login();

In [ ]:
# Load Data 
studies = create_summary.getStudies(syn)

def readCompletionFile(synId, version=None):
    df = pd.read_csv(syn.get(synId, version=version).path, low_memory=False)
    timeCols =  ['uploadedOn', 'finishedOn', 'startedOn', 'eventTimestamp_x', 'modifiedOn', 'createdOn', 'exportedOn', 'eventTimestamp_y']
    df[timeCols] = df[timeCols].apply(pd.to_datetime)

    #Fill in startedOn data for columns and remove fnamea
    df = (df.assign(startedOn = np.where(df.startedOn.isnull(), df.uploadedOn, df.startedOn))
          .query("assessmentId!='fnamea'")
          .set_index('startedOn', drop=True))
    return df

df = readCompletionFile('syn51185576') 
df['completion time [s]'] = (df.finishedOn-df.index).map(lambda x:x.seconds)
df['export delay [s]'] = (df.exportedOn-df.finishedOn).map(lambda x:x.seconds)
df['upload delay [s]'] = (df.uploadedOn-df.finishedOn).map(lambda x:x.seconds)
df['in Synapse delay [s]'] = (df.createdOn-df.finishedOn).map(lambda x:x.seconds)


# Percent complete Data
Percent of data that was reported as completed by adherence tracking in Bridge that has made it to different stages in the pipeline.$^{1}$ Number of records at each stage is reported by n.

$^{1.}$<font size="1">Percents can be higher than 100 if the data was uploaded but the adherence records was not uploaded to Bridge.</font>

In [ ]:
temp_df = df.dropna(subset=['wasCompleted'])[COMPLETION_COLS]
n_alltime = temp_df.sum()
pct_alltime = pd.DataFrame(n_alltime.div(n_alltime['wasCompleted'], axis=0)*100, columns=['All Time'])
labels_alltime = pd.DataFrame([f"{a:.0f}%\n\nn={b:,.0f}" for a, b in zip(pct_alltime['All Time'], n_alltime)])

plt.figure(figsize=(13,2.5))
#ax = sns.heatmap(pct_alltime.T, annot=True, fmt=".0f", cbar=False, linewidth=5, vmin=0);
ax = sns.heatmap(pct_alltime.T, annot=labels_alltime.T, fmt="s", cbar=False, linewidth=5, vmin=0,
                annot_kws={"fontsize":22});
plt.yticks([])
plt.title('All Time', fontsize=22);

In [ ]:
n_lastweek = temp_df.groupby(pd.Grouper(freq='W')).sum().iloc[-2]
legend = n_lastweek.name.strftime('week ending %d-%b-%Y')
n_lastweek.name = legend
pct_lastweek  = pd.DataFrame(n_lastweek.div(n_lastweek['wasCompleted'], axis=0)*100)
labels_lastweek = pd.DataFrame([f"{a:.0f}%\n\nn={b:,.0f}" for a, b in zip(pct_lastweek[legend], n_lastweek)])

#n_lastweek
plt.figure(figsize=(13,2.5))
#ax = sns.heatmap(pct_lastweek.T, annot=True, fmt=".0f", cbar=False, linewidth=5,vmin=0);
ax = sns.heatmap(pct_lastweek.T, annot=labels_lastweek.T, fmt="s", cbar=False, linewidth=5, vmin=0,
                annot_kws={"fontsize":22});
plt.yticks([])
plt.title(legend, fontsize=22);

In [ ]:
n_lastweek = temp_df.groupby(pd.Grouper(freq='W')).sum().iloc[-1]
legend = n_lastweek.name.strftime('week ending %d-%b-%Y')
n_lastweek.name = legend
pct_lastweek  = pd.DataFrame(n_lastweek.div(n_lastweek['wasCompleted'], axis=0)*100)
labels_lastweek = pd.DataFrame([f"{a:.0f}%\n\nn={b:,.0f}" for a, b in zip(pct_lastweek[legend], n_lastweek)])

#n_lastweek
plt.figure(figsize=(13,2.5))
#ax = sns.heatmap(pct_lastweek.T, annot=True, fmt=".0f", cbar=False, linewidth=5,vmin=0);
ax = sns.heatmap(pct_lastweek.T, annot=labels_lastweek.T, fmt="s", cbar=False, linewidth=5, vmin=0,
                annot_kws={"fontsize":22});
plt.yticks([])
plt.title(legend, fontsize=22);

### Last Scored data by Study

In [ ]:
(studies[['studyId', 'name']]
 .merge(df.query("inScores==1").groupby('studyId')['finishedOn'].max(), on='studyId')
).set_index('studyId').sort_index()



In [ ]:
fig = plt.figure(figsize=(12,5))

ax2 = plt.subplot2grid((4, 1), (0, 0), rowspan=1, colspan=1)
(df.query("wasCompleted==1").groupby(pd.Grouper(freq='W'))['wasCompleted'].count()).plot(ax=ax2, kind='bar')
plt.ylabel('Nr Records');
plt.title('Data Status Across Time')
plt.xticks([]);
plt.xlabel('')
ax1 = plt.subplot2grid((4, 1), (1, 0), rowspan=3, colspan=1)
(df.query("wasCompleted==1").groupby(pd.Grouper(freq='W'))[COMPLETION_COLS].mean()*100).plot(ax=ax1)
plt.ylim(0,102)
plt.ylabel('completed [%]');
ticks, labels = plt.xticks()

### Changes in processing across time

The following graphs looks at the state of the data as it appeared at different times since March-2023.  That is, as improvements have been made and data recovered the counts have changed.  This uses historical snapshots stored in versions of [syn51185576](https://www.synapse.org/#!Synapse:syn51185576) to infer counts of records and percentages.  This is currently done for data collected across all time and data collected since January-2023.

In [ ]:
def summarizeCompletion(df):
    """Summarized state of completion for a specific dataframe.
    
    Args
        df - completion data frame
        
    returns - data frame with date column label and rows with summaries.
    """
    #Only look at records where we have a completion record
    df = df.dropna(subset=['wasCompleted'])[COMPLETION_COLS]

    date = df.index.max()
    n_alltime = df.sum()
    pct_alltime = pd.DataFrame(n_alltime.div(n_alltime['wasCompleted'], axis=0)*100, columns=['All Time'])
    #Summarize since jan-1-2023
    df = df[df.index>'2023']
    n_2023 = df.sum()
    pct_2023 = pd.DataFrame(n_2023.div(n_2023['wasCompleted'], axis=0)*100, columns=['All Time'])
    return pd.concat([n_alltime, pct_alltime['All Time'], n_2023, pct_2023['All Time']], 
               keys=[('beginning', 'N'), ('beginning','pct'), ('2023', 'N'), ('2023','pct')], 
               names=['timeFrame','summary']).to_frame(name=date)
    
    
maxVersion = syn.get('syn51185576', downloadFile=False).versionNumber
dfs = [readCompletionFile('syn51185576', version) 
       for version in range(7, maxVersion+1)]
summary = pd.concat([summarizeCompletion(df) for df in dfs], axis=1).T

plt.figure(figsize=(13,4))
for i, t in enumerate(['beginning', '2023']):
    ax = plt.subplot(1,2,i+1)
    (summary[[(t, 'pct','inParquet'), (t, 'pct','inScores'),(t, 'pct','inStudyProject')]]
        .droplevel([0,1], axis=1)
        .plot(ax=ax))
    plt.ylabel('Percent available')
    plt.xlabel('Date scored')
    plt.title(f'Data generated since {t}')
    plt.grid(True)

# Associations between missing data and conditions when collected
Evaluates whether there are any specific conditions that lead to a higher risk of missing data. For example whether the appVersion or the sessioon is assoicated with data being missing in the **scores** stage. Can also be modified to identify other missingness for example in upload or parquet data. **Only looks at data from 'completed' assessments, i.e those that weren't declined.**

In [ ]:

#Columns that are evaluated
correlated_columns=['wasCompleted','assessmentId', 'studyId', 'appVersion','assessmentRevision', 'clientTimeZone', 'declined', 
                    'deviceName', 'languages','osName', 'osVersion', 'scheduleGuid', 'sessionGuid', 
                    'sessionInstanceGuid', 'sessionInstanceStartDay', 'sessionStartEventId']

@widgets.interact(stage_missing=['inScores', 'inParquet', 'inStudyProject'], 
                  end_date = df.index.max().strftime('%Y-%m-%d %X'),
                  p_cut_off=widgets.fixed(1e-3))
def find_correlated_variables(start_date='2023-03-01 00:00:00', end_date='2023-04-01 00:00:00', stage_missing='inScores', p_cut_off=0.001):
    """Filters the data by timeframe and compares the differences in missingness in a specific stage in
       pipeline.  Computes contigency tables and determines if differences are statistically significant and plot
       the odds of missing data.
       
    """
    temp_df = df[df.index > start_date].query('wasCompleted==1')
    temp_df = temp_df[temp_df.index<end_date]
    nMissingSynapse = sum(temp_df['inStudyProject']==0)
    nMissingParquet = sum(temp_df['inParquet']==0)
    nMissingScores = sum(temp_df['inScores']==0)
    print(f'Between {start_date} and {end_date}: {nMissingScores} ({100*nMissingScores/len(temp_df):0.2f}%) missing out of {len(temp_df)}')
    print(f'{nMissingSynapse} ({100*nMissingSynapse/len(temp_df):0.2f}%) records missing in Synapse')
    print(f'{nMissingParquet-nMissingSynapse} ({100*(nMissingParquet-nMissingSynapse)/len(temp_df):0.2f}%) additional missing in Parquet')
    print(f'{nMissingScores-nMissingParquet} ({100*(nMissingScores-nMissingParquet)/len(temp_df):0.2f}%) additional missing from Scores')
    
    #Filter out the previous stage so we only focus on data missing between the previous stage and the "stage_missing"
    if stage_missing == 'inParquet':
        temp_df = temp_df.query("inStudyProject==1")
    elif stage_missing == 'inScores':
        temp_df = temp_df.query("inParquet==1")

    for variable in correlated_columns:
        #Create contigency table
        contigency = (temp_df[[stage_missing, variable]]
             .groupby(variable)[stage_missing]
             .value_counts()  #Count number of missing and nn missing
             .to_frame(name='available')      
             .reset_index(1)
             .pivot_table(columns=stage_missing, values='available', index=variable)
             .fillna(0)  #If there are no counts set the NaN to 0
             .rename(columns={1.0:'Present', 0.0:'Missing'})
        )
        # Filtering out any categories with too few measurements 
        contigency = contigency[contigency.sum(axis=1)>10]
        # Only continue of data is missing
        if 'Missing' not in contigency.columns:
            continue
        # Compute Chi-square statistic for missingness being different between groups
        # Assumption there is no relationship between variable and missingness
        chi2, p, dof, expected = stats.chi2_contingency(contigency)
    
        # odds of missing data = (# 0s)/(# 1s)
        contigency['Odds']=contigency['Missing']/contigency['Present']
    
        #Plot odds of missingness for each category if the p-value<0.001
        if p<p_cut_off:
            plt.figure(figsize=(13,6))
            ax=plt.subplot(2,1,1)
            contigency.plot(y='Odds', kind='bar', ax=ax);
            plt.ylabel('Odds of Missing')
            plt.title('%s p=%0.2g' %(variable, p))
            ax=plt.subplot(2,1,2)
            #Format table for display
            contigency['Odds'] = contigency['Odds'].apply('{:,.1f}'.format)
            contigency = contigency.astype({'Present': int, 'Missing': int})
            plt.table(cellText=contigency.values.T, rowLabels=contigency.columns, loc='center')
            plt.axis('off');

# Study Specific information
Filter down to a specific study and explore the data within that study

In [ ]:
groups = df.groupby('studyId')
@widgets.interact(study=groups.groups.keys())
def plot_studies(study):
    plt.figure(figsize=(2,8))
    temp_df = groups.get_group(study)[COMPLETION_COLS]
    ax = plt.subplot(1,1,1)
    sns.heatmap(temp_df, cbar=False, ax=ax)
    ticklabels = ['NaT' if pd.isnull(temp_df.index[int(tick)]) else temp_df.index[int(tick)].strftime('%Y-%m-%d') for tick in ax.get_yticks()]
    ax.set_yticklabels(ticklabels);
    label = (studies.query("studyId=='%s'"%study)['name']+'('+ studies.query("studyId=='%s'"%study)['id'] + ')').iloc[0]
    plt.title(label)

In [ ]:
@widgets.interact(study=groups.groups.keys(), stages=COMPLETION_COLS )
def show_participant_level_data(study='hktrrx', stages='inStudyProject'):
    return (df.query("studyId=='%s'" %study)
           .groupby(['externalId', 'sessionGuid'])[stages]
           .apply(sum).reset_index()
           .pivot(index='externalId', columns='sessionGuid')
           .fillna(0)
           .astype(int))
    

# Timing information

In [ ]:
plt.figure(figsize=(8,6))
    
timing = df.groupby(pd.Grouper(freq='W'))[DELTA_T_COLS].median()
sns.lineplot(data=timing)
plt.ylabel('Delay [s]');
plt.yscale('log')
timing = timing.reset_index().melt(id_vars=['startedOn'])

# Apps in active use
App Versions being used across time.

In [ ]:
oses = ['Android', 'iPadOS', 'iPhone OS']
# Combine and count appVersions by week
appCounts = (df.dropna(subset=['wasCompleted'])
             .groupby([pd.Grouper(freq='W'),'osName', 'appVersion'])[['healthCode']]
             .nunique()
             .reset_index())
appCounts = appCounts.assign(penetration = appCounts.healthCode/appCounts.groupby(['startedOn', 'osName']).healthCode.transform('sum'))
appCounts = appCounts.pivot_table(index='startedOn', columns=['osName', 'appVersion'], values='penetration').fillna(0)

# Plot and outoput last week 
plt.figure(figsize=(13,13))
for i, os in enumerate(oses):
    ax=plt.subplot(3,1,i+1)
    appCounts.loc[:, os].plot(ax=ax)
    plt.title(os)
print('This weeks usage in percent by OS')
#with pd.option_context('display.float_format', '{:,.2f}%'.format):
last_week = appCounts.loc[:, ['Android', 'iPhone OS']].iloc[-1:]
(last_week.T[last_week.T!=0].dropna()*100).round()


## Number of Active Users

In [ ]:
plt.figure(figsize=(13,5))
userCount = df.groupby(pd.Grouper(freq='M'))['healthCode'].nunique()
userCount.plot()
plt.title('Nr Active users by month');
print('Number of users in this month:', userCount[-1])
